In [91]:
import pandas as pd
import pyGPs
import numpy as np
import math
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import metrics
from scipy import stats
import scipy.integrate as integrate
import scipy.spatial.distance as spdist
import logging
from bs4 import BeautifulSoup, Comment
import requests
import re
import sys
import operator

In [212]:
df = pd.read_csv("2017games.csv")
df.drop(columns=['StartTime', 'Unnamed: 6', 'Unnamed: 7', 'Notes', 'Attendance'], inplace=True)
df["MarginHome"] = df["PointsHome"] - df["PointsAway"]
df

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12


In [ ]:
df = pd.read_csv("2017games.csv")
df.drop(columns=['StartTime', 'Unnamed: 6', 'Unnamed: 7', 'Notes', 'Attendance'], inplace=True)
df["MarginHome"] = df["PointsHome"] - df["PointsAway"]
df

In [213]:
teamDict = {
    "Cleveland Cavaliers":"CLE",
    "Golden State Warriors":"GSW",
    "Portland Trail Blazers":"POR",
    "Boston Celtics":"BOS",
    "Indiana Pacers":"IND",
    "Los Angeles Lakers":"LAL",
    "Memphis Grizzlies":"MEM",
    "Milwaukee Bucks":"MIL",
    "New Orleans Pelicans":"NOP",
    "Orlando Magic":"ORL",
    "Philadelphia 76ers":"PHI",
    "Phoenix Suns":"PHO",
    "Toronto Raptors":"TOR",
    "Atlanta Hawks":"ATL",
    "Chicago Bulls":"CHI",
    "Sacramento Kings":"SAC",
    "Brooklyn Nets":"BRK",
    "Dallas Mavericks":"DAL",
    "Detroit Pistons":"DET",
    "Miami Heat":"MIA",
    "Oklahoma City Thunder":"OKC",
    "Utah Jazz":"UTA",
    "Charlotte Hornets":"CHO",
    "Denver Nuggets":"DEN",
    "New York Knicks":"NYK",
    "Houston Rockets":"HOU",
    "Washington Wizards":"WAS",
    "San Antonio Spurs":"SAS",
    "Minnesota Timberwolves":"MIN",
    "Los Angeles Clippers":"LAC"
}
monthNum = {"Oct":"10", "Nov":"11", "Dec":"12", "Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04"}
links = []
for j in df.iterrows():
    date = j[1]["Date"]
    team = j[1]["TeamHome"]
    year = date[len(date)-4:]
    month = monthNum[date[4:7]]
    day = date[len(date)-7:len(date)-5]
    if ' ' in day:
        day = '0' + day[1]
    day = day + '0'
    abbrev = teamDict[team]
    link = year + month + day + abbrev
    links.append(link)
df['Links'] = links
df

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Links
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,201610250CLE
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,201610250GSW
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,201610250POR
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,201610260BOS
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,201610260IND
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,201610260LAL
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,201610260MEM
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,201610260MIL
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,201610260NOP
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,201610260ORL


In [219]:
for index, i in df.iterrows():
    result = "Loss"
    if i.MarginHome > 0:
        result = "Win"
    df.at[index,"Result"] = result
df

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Links,Result
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,201610250CLE,Win
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,201610250GSW,Loss
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,201610250POR,Win
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,201610260BOS,Win
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,201610260IND,Win
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,201610260LAL,Win
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,201610260MEM,Win
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,201610260MIL,Loss
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,201610260NOP,Loss
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,201610260ORL,Loss


In [222]:
for i, link in enumerate(df['Links']):
    if i % 10 == 0:
        print(i)
    url = "https://www.basketball-reference.com/boxscores/" + link + ".html"
    request = requests.get(url)
    if request.status_code != 200:
        print("ERROR")
        print(link)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    comments = soup.find_all(string=lambda text:isinstance(text,Comment))
    for comment in comments:
        comment = BeautifulSoup(str(comment), 'lxml')
        tableFF = comment.find('table', id='four_factors')
        if tableFF:
            break
    all_trs = tableFF.find_all('tr')
    awayTR = all_trs[-2]
    homeTR = all_trs[-1]
    all_tds = awayTR.find_all('td')
    for td in all_tds:
        if td['data-stat'] == 'pace':
            df.at[i, 'Pace'] = float(td.text)
        if td['data-stat'] == 'efg_pct':
            df.at[i, 'aEFG'] = float(td.text) * 100
        if td['data-stat'] == 'tov_pct':
            df.at[i, 'aTOV'] = float(td.text)
        if td['data-stat'] == 'orb_pct':
            df.at[i, 'aORB'] = float(td.text)
        if td['data-stat'] == 'ft_rate':
            df.at[i, 'aFTR'] = float(td.text) * 100
    all_tds = homeTR.find_all('td')
    for td in all_tds:
        if td['data-stat'] == 'efg_pct':
            df.at[i, 'hEFG'] = float(td.text) * 100
        if td['data-stat'] == 'tov_pct':
            df.at[i, 'hTOV'] = float(td.text)
        if td['data-stat'] == 'orb_pct':
            df.at[i, 'hORB'] = float(td.text)
        if td['data-stat'] == 'ft_rate':
            df.at[i, 'hFTR'] = float(td.text) * 100
df

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220


,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Links,Result,Pace,aEFG,aTOV,aORB,aFTR,hEFG,hTOV,hORB,hFTR
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,201610250CLE,Win,99.9,42.0,15.8,24.5,17.2,54.8,12.0,27.5,14.9
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,201610250GSW,Loss,98.3,54.1,10.6,43.8,23.5,51.2,14.7,19.0,15.3
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,201610250POR,Win,90.5,53.7,11.0,17.1,19.5,60.7,12.4,16.7,29.3
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,201610260BOS,Win,103.0,52.1,13.1,30.0,16.5,60.1,16.4,29.3,16.9
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,201610260IND,Win,103.6,51.9,11.8,18.5,12.5,55.9,12.9,17.0,28.0
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,201610260LAL,Win,99.4,55.2,12.5,26.3,22.1,57.3,13.2,27.5,20.2
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,201610260MEM,Win,91.2,45.3,9.2,30.6,23.3,51.9,12.4,26.8,23.8
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,201610260MIL,Loss,96.5,48.9,9.1,23.4,22.5,44.0,7.4,20.0,16.3
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,201610260NOP,Loss,107.1,49.4,19.7,28.2,30.1,42.9,9.6,9.6,25.0
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,201610260ORL,Loss,94.3,50.5,8.8,34.8,10.3,41.6,9.8,29.4,24.7


In [271]:
predDF = pd.read_csv("2017games.csv")
predDF.drop(columns=['StartTime', 'Unnamed: 6', 'Unnamed: 7', 'Notes', 'Attendance'], inplace=True)
predDF["MarginHome"] = predDF["PointsHome"] - predDF["PointsAway"]
predDF["Result"] = df["Result"]
predDF

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Result
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,Win
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,Loss
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,Win
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,Win
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,Win
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,Win
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,Win
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,Loss
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,Loss
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,Loss


In [235]:
teams = []
for i in df['TeamHome']:
    if i not in teams:
        teams.append(i)
teams.sort()
teamsDF = pd.DataFrame(teams)
teamsDF.columns = ['Team']
teamsDF['G'] = 0
teamsDF['oEFG'] = 0.0
teamsDF['oTOV'] = 0.0
teamsDF['ORB'] = 0.0
teamsDF['oFTR'] = 0.0
teamsDF['dEFG'] = 0.0
teamsDF['dTOV'] = 0.0
teamsDF['DRB'] = 0.0
teamsDF['dFTR'] = 0.0
teamsDF

,Team,G,oEFG,oTOV,ORB,oFTR,dEFG,dTOV,DRB,dFTR
0,Atlanta Hawks,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Boston Celtics,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brooklyn Nets,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Charlotte Hornets,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Chicago Bulls,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Cleveland Cavaliers,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Dallas Mavericks,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Denver Nuggets,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Detroit Pistons,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Golden State Warriors,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
prevDate = ''
x = []
margins = []
k = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model = pyGPs.GPR()
model.setPrior(mean=pyGPs.mean.Zero(), kernel=k)
modelSet = False
for date in df.Date:
    if date != prevDate:
        print(date)
        games = df.loc[df['Date'] == date]
        # create x and y
        for index, game in games.iterrows():
            aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
            aTeamOff = aTeam[0][2:6]
            aTeamDef = aTeam[0][6:]
            hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
            hTeamOff = hTeam[0][2:6]
            hTeamDef = hTeam[0][6:]
#             hOffDef = list(map(operator.sub, hTeamOff, aTeamDef))
#             hDefOff = list(map(operator.sub, hTeamDef, aTeamOff))
#             x.append(hOffDef + hDefOff)
            hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
            aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
            netOffDef = list(map(operator.sub, hOffDef, aOffDef))
            x.append(netOffDef)
            margins.append(int(game.MarginHome))
        # store predictions
        for index, game in games.iterrows():
            aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
            aTeamOff = aTeam[0][2:6]
            aTeamDef = aTeam[0][6:]
            hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
            hTeamOff = hTeam[0][2:6]
            hTeamDef = hTeam[0][6:]
#             hOffDef = list(map(operator.sub, hTeamOff, aTeamDef))
#             hDefOff = list(map(operator.sub, hTeamDef, aTeamOff))
            hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
            aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
            netOffDef = list(map(operator.sub, hOffDef, aOffDef))
            if modelSet:
                ym, ys2, fmu, fs2, lp = model.predict(np.array([netOffDef]))
            else:
                ym = 0
                ys2 = 0
                lp = 0
            predDF.at[index,'PredMean'] = ym
            predDF.at[index,'PredVar'] = ys2
            predDF.at[index,'PredProb'] = lp
        # fit model
#         print(np.array(x))
        model.getPosterior(np.array(x), np.array(margins), False)
        modelSet = True
        # update teamsDF
        for index, game in games.iterrows():
            aTeam = game.TeamAway
            aGames = teamsDF.loc[teamsDF['Team'] == aTeam,'G']

            teamsDF.loc[teamsDF['Team'] == aTeam,'oEFG'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'oEFG'] += game.aEFG
            teamsDF.loc[teamsDF['Team'] == aTeam,'oEFG'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'oTOV'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'oTOV'] += game.aTOV
            teamsDF.loc[teamsDF['Team'] == aTeam,'oTOV'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'ORB'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'ORB'] += game.aORB
            teamsDF.loc[teamsDF['Team'] == aTeam,'ORB'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'oFTR'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'oFTR'] += game.aFTR
            teamsDF.loc[teamsDF['Team'] == aTeam,'oFTR'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'dEFG'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'dEFG'] += game.hEFG
            teamsDF.loc[teamsDF['Team'] == aTeam,'dEFG'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'dTOV'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'dTOV'] += game.hTOV
            teamsDF.loc[teamsDF['Team'] == aTeam,'dTOV'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'DRB'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'DRB'] += game.hORB
            teamsDF.loc[teamsDF['Team'] == aTeam,'DRB'] /= (aGames + 1)
            teamsDF.loc[teamsDF['Team'] == aTeam,'dFTR'] *= aGames
            teamsDF.loc[teamsDF['Team'] == aTeam,'dFTR'] += game.hFTR
            teamsDF.loc[teamsDF['Team'] == aTeam,'dFTR'] /= (aGames + 1)

            hTeam = game.TeamHome
            hGames = teamsDF.loc[teamsDF['Team'] == hTeam,'G']

            teamsDF.loc[teamsDF['Team'] == hTeam,'oEFG'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'oEFG'] += game.hEFG
            teamsDF.loc[teamsDF['Team'] == hTeam,'oEFG'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'oTOV'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'oTOV'] += game.hTOV
            teamsDF.loc[teamsDF['Team'] == hTeam,'oTOV'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'ORB'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'ORB'] += game.hORB
            teamsDF.loc[teamsDF['Team'] == hTeam,'ORB'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'oFTR'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'oFTR'] += game.hFTR
            teamsDF.loc[teamsDF['Team'] == hTeam,'oFTR'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'dEFG'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'dEFG'] += game.aEFG
            teamsDF.loc[teamsDF['Team'] == hTeam,'dEFG'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'dTOV'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'dTOV'] += game.aTOV
            teamsDF.loc[teamsDF['Team'] == hTeam,'dTOV'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'DRB'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'DRB'] += game.aORB
            teamsDF.loc[teamsDF['Team'] == hTeam,'DRB'] /= (hGames + 1)
            teamsDF.loc[teamsDF['Team'] == hTeam,'dFTR'] *= hGames
            teamsDF.loc[teamsDF['Team'] == hTeam,'dFTR'] += game.aFTR
            teamsDF.loc[teamsDF['Team'] == hTeam,'dFTR'] /= (hGames + 1)
            
            teamsDF.loc[teamsDF['Team'] == aTeam,'G'] += 1
            teamsDF.loc[teamsDF['Team'] == hTeam,'G'] += 1
    prevDate = date  
predDF

Tue Oct 25 2016
Wed Oct 26 2016
Thu Oct 27 2016
Fri Oct 28 2016
Sat Oct 29 2016
Sun Oct 30 2016
Mon Oct 31 2016
Tue Nov 1 2016
Wed Nov 2 2016
Thu Nov 3 2016
Fri Nov 4 2016
Sat Nov 5 2016
Sun Nov 6 2016
Mon Nov 7 2016
Tue Nov 8 2016
Wed Nov 9 2016
Thu Nov 10 2016
Fri Nov 11 2016
Sat Nov 12 2016
Sun Nov 13 2016
Mon Nov 14 2016
Tue Nov 15 2016
Wed Nov 16 2016
Thu Nov 17 2016
Fri Nov 18 2016
Sat Nov 19 2016
Sun Nov 20 2016
Mon Nov 21 2016
Tue Nov 22 2016
Wed Nov 23 2016
Fri Nov 25 2016
Sat Nov 26 2016
Sun Nov 27 2016
Mon Nov 28 2016
Tue Nov 29 2016
Wed Nov 30 2016
Thu Dec 1 2016
Fri Dec 2 2016
Sat Dec 3 2016
Sun Dec 4 2016
Mon Dec 5 2016
Tue Dec 6 2016
Wed Dec 7 2016
Thu Dec 8 2016
Fri Dec 9 2016
Sat Dec 10 2016
Sun Dec 11 2016
Mon Dec 12 2016
Tue Dec 13 2016
Wed Dec 14 2016
Thu Dec 15 2016
Fri Dec 16 2016
Sat Dec 17 2016
Sun Dec 18 2016
Mon Dec 19 2016
Tue Dec 20 2016
Wed Dec 21 2016
Thu Dec 22 2016
Fri Dec 23 2016
Sun Dec 25 2016
Mon Dec 26 2016
Tue Dec 27 2016
Wed Dec 28 2016
Thu Dec 29

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Result,PredMean,PredVar,PredProb
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,Win,0.000000e+00,0.000000,0.0
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,Loss,0.000000e+00,0.000000,0.0
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,Win,0.000000e+00,0.000000,0.0
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,Win,2.990033e+00,0.013322,NaN
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,Win,2.990033e+00,0.013322,NaN
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,Win,2.990033e+00,0.013322,NaN
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,Win,2.990033e+00,0.013322,NaN
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,Loss,2.990033e+00,0.013322,NaN
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,Loss,2.990033e+00,0.013322,NaN
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,Loss,2.990033e+00,0.013322,NaN


In [274]:
numRight = 0
for index, row in predDF.iterrows():
    margin = row.MarginHome
    pred = row.PredMean
    if margin > 0 and pred > 0:
        numRight += 1
    if margin < 0 and pred < 0:
        numRight += 1
#     print(pred)
numRight / len(predDF)

0.8772357723577235

In [209]:
model2 = pyGPs.GPR()
model2.setPrior(mean=pyGPs.mean.Zero(), kernel=k)
model2.optimize(np.array(x),np.array(margins))

Number of line searches 40


In [5]:
x = np.array([-2.5,-1.5,-0.5,0.5,1.5,2.5]).T
y = np.array([math.exp(-(i**2)) for i in x]).T

k1 = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model = pyGPs.GPR()
model.setPrior(mean=pyGPs.mean.Zero(), kernel=k1)
nlz, post = model.getPosterior(x, y, False)
nlz

5.604608563078781

In [254]:
x = np.array([-2.5,-1.5,-0.5,0.5,1.5,2.5]).T
y = np.array([-1, -1, -1, 1, 1, 1]).T

k1 = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model = pyGPs.GPC()
model.setPrior(mean=pyGPs.mean.Zero(), kernel=k1)
# xs = np.array([])
# ys = np.array([])
# for i in x:
#     xs = np.append(xs, i)
#     ys = np.append(ys, math.exp(-(i**2)))
#     print(xs)
#     print(ys)
#     nlz, post = model.getPosterior(xs, ys, False)
#     print(nlz)
#     print()
nlz, post = model.getPosterior(x, y, False)
print(nlz)
print()
model.predict(np.array([2.51]))

3.8365139118616898



(array([[0.39954364]]),
 array([[0.84036488]]),
 array([[0.67824039]]),
 array([[0.67698273]]),
 None)

In [ ]:
paces = []
EFGs = []
TOVs = []
ORBs = []
FTRs = []
for j in df.iterrows():
#     print(j)
    teamAway = j[1]["TeamAway"]
    teamHome = j[1]["TeamHome"]
    if teamAway == "Golden State Warriors":
        paces.append(j[1]["Pace"])
        EFGs.append(j[1]["aEFG"])
        TOVs.append(j[1]["aTOV"])
        ORBs.append(j[1]["aORB"])
        FTRs.append(j[1]["aFTR"])
    if teamHome == "Golden State Warriors":
        paces.append(j[1]["Pace"])
        EFGs.append(j[1]["hEFG"])
        TOVs.append(j[1]["hTOV"])
        ORBs.append(j[1]["hORB"])
        FTRs.append(j[1]["hFTR"])
print("Avg Pace:", np.mean(paces))
print("Avg EFGs:", np.mean(EFGs))
print("Avg TOVs:", np.mean(TOVs))
print("Avg ORBs:", np.mean(ORBs))
print("Avg FTRs:", np.mean(FTRs))

In [278]:
prevDate = ''
x = []
margins = []
k = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model = pyGPs.GPR()
model.setPrior(mean=pyGPs.mean.Zero(), kernel=k)
modelSet = False
for date in df.Date:
    if date != prevDate:
        print(date)
        games = df.loc[df['Date'] == date]
        # create x and y
        for index, game in games.iterrows():
            aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
            aTeamOff = aTeam[0][2:6]
            aTeamDef = aTeam[0][6:]
            hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
            hTeamOff = hTeam[0][2:6]
            hTeamDef = hTeam[0][6:]
            hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
            aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
            netOffDef = list(map(operator.sub, hOffDef, aOffDef))
            x.append(netOffDef)
            margins.append(int(np.sign(game.MarginHome)))
        # store predictions
        for index, game in games.iterrows():
            aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
            aTeamOff = aTeam[0][2:6]
            aTeamDef = aTeam[0][6:]
            hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
            hTeamOff = hTeam[0][2:6]
            hTeamDef = hTeam[0][6:]
            hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
            aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
            netOffDef = list(map(operator.sub, hOffDef, aOffDef))
            if modelSet:
                ym, ys2, fmu, fs2, lp = model.predict(np.array([netOffDef]))
            else:
                ym = 0
                ys2 = 0
                lp = 0
            predDF.at[index,'PredMean'] = ym
            predDF.at[index,'PredVar'] = ys2
            predDF.at[index,'PredProb'] = lp
        # fit model
#         print(np.array(x))
        model.getPosterior(np.array(x), np.array(margins), False)
        modelSet = True
    prevDate = date  
predDF

Tue Oct 25 2016
Wed Oct 26 2016
Thu Oct 27 2016
Fri Oct 28 2016
Sat Oct 29 2016
Sun Oct 30 2016
Mon Oct 31 2016
Tue Nov 1 2016
Wed Nov 2 2016
Thu Nov 3 2016
Fri Nov 4 2016
Sat Nov 5 2016
Sun Nov 6 2016
Mon Nov 7 2016
Tue Nov 8 2016
Wed Nov 9 2016
Thu Nov 10 2016
Fri Nov 11 2016
Sat Nov 12 2016
Sun Nov 13 2016
Mon Nov 14 2016
Tue Nov 15 2016
Wed Nov 16 2016
Thu Nov 17 2016
Fri Nov 18 2016
Sat Nov 19 2016
Sun Nov 20 2016
Mon Nov 21 2016
Tue Nov 22 2016
Wed Nov 23 2016
Fri Nov 25 2016
Sat Nov 26 2016
Sun Nov 27 2016
Mon Nov 28 2016
Tue Nov 29 2016
Wed Nov 30 2016
Thu Dec 1 2016
Fri Dec 2 2016
Sat Dec 3 2016
Sun Dec 4 2016
Mon Dec 5 2016
Tue Dec 6 2016
Wed Dec 7 2016
Thu Dec 8 2016
Fri Dec 9 2016
Sat Dec 10 2016
Sun Dec 11 2016
Mon Dec 12 2016
Tue Dec 13 2016
Wed Dec 14 2016
Thu Dec 15 2016
Fri Dec 16 2016
Sat Dec 17 2016
Sun Dec 18 2016
Mon Dec 19 2016
Tue Dec 20 2016
Wed Dec 21 2016
Thu Dec 22 2016
Fri Dec 23 2016
Sun Dec 25 2016
Mon Dec 26 2016
Tue Dec 27 2016
Wed Dec 28 2016
Thu Dec 29

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Result,PredMean,PredVar,PredProb
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,Win,0.000000e+00,0.000000,0.0
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,Loss,0.000000e+00,0.000000,0.0
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,Win,0.000000e+00,0.000000,0.0
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,Win,3.480448e-21,1.010000,NaN
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,Win,4.514812e-14,1.010000,NaN
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,Win,4.836818e-13,1.010000,NaN
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,Win,1.563489e-05,1.010000,NaN
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,Loss,-3.695517e-17,1.010000,NaN
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,Loss,-1.197085e-34,1.010000,NaN
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,Loss,2.170656e-26,1.010000,NaN


In [280]:
numRight = 0
for index, row in predDF.iterrows():
    margin = row.MarginHome
    pred = row.PredMean
    if margin > 0 and pred > 0:
        numRight += 1
    if margin < 0 and pred < 0:
        numRight += 1
#     print(pred)
numRight / len(predDF)

0.5552845528455285

In [311]:
x = []
margins = []
k = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model = pyGPs.GPR()
model.setPrior(mean=pyGPs.mean.Zero(), kernel=k)
modelSet = False
# create x and y
for index, game in df.iterrows():
    aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
    aTeamOff = aTeam[0][2:6]
    aTeamDef = aTeam[0][6:]
    hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
    hTeamOff = hTeam[0][2:6]
    hTeamDef = hTeam[0][6:]
    hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
    aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
    netOffDef = list(map(operator.sub, hOffDef, aOffDef))
    x.append(netOffDef)
    margins.append(int(game.MarginHome))
# store predictions
model.getPosterior(np.array(x), np.array(margins), False)
for index, game in df.iterrows():
    if index % 50 == 0:
        print(index)
    aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
    aTeamOff = aTeam[0][2:6]
    aTeamDef = aTeam[0][6:]
    hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
    hTeamOff = hTeam[0][2:6]
    hTeamDef = hTeam[0][6:]
    hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
    aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
    netOffDef = list(map(operator.sub, hOffDef, aOffDef))
    ym, ys2, fmu, fs2, lp = model.predict(np.array([netOffDef]))
    predDF.at[index,'PredMean'] = ym[0]
    predDF.at[index,'PredVar'] = ys2[0]
predDF

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200


,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Result,PredMean,PredVar,PredProb
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,29,Win,21.895296,0.014974,0.0
1,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,-29,Loss,-28.695549,0.019897,0.0
2,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,9,Win,11.976844,0.014972,0.0
3,Wed Oct 26 2016,Brooklyn Nets,117,Boston Celtics,122,5,Win,6.963845,0.014975,NaN
4,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,9,Win,8.899915,0.019895,NaN
5,Wed Oct 26 2016,Houston Rockets,114,Los Angeles Lakers,120,6,Win,5.902623,0.019890,NaN
6,Wed Oct 26 2016,Minnesota Timberwolves,98,Memphis Grizzlies,102,4,Win,12.935956,0.014975,NaN
7,Wed Oct 26 2016,Charlotte Hornets,107,Milwaukee Bucks,96,-11,Loss,-0.479837,0.014973,NaN
8,Wed Oct 26 2016,Denver Nuggets,107,New Orleans Pelicans,102,-5,Loss,-3.982208,0.014975,NaN
9,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,-12,Loss,-0.496956,0.014975,NaN


In [296]:
teamsDF

,Team,G,oEFG,oTOV,ORB,oFTR,dEFG,dTOV,DRB,dFTR
0,Atlanta Hawks,82,50.531707,13.808537,23.502439,21.706098,50.789024,13.297561,23.669512,18.000000
1,Boston Celtics,82,52.579268,11.692683,20.967073,22.182927,50.398780,12.167073,24.484146,22.552439
2,Brooklyn Nets,82,50.876829,14.253659,19.242683,23.106098,51.496341,11.215854,23.430488,21.447561
3,Charlotte Hornets,82,50.217073,10.240244,19.704878,22.942683,52.557317,11.674390,20.197561,16.512195
4,Chicago Bulls,82,48.817073,11.804878,26.836585,20.857317,50.841463,11.950000,23.123171,17.073171
5,Cleveland Cavaliers,83,54.750602,12.167470,21.886747,20.644578,51.580723,10.996386,24.172289,17.568675
6,Dallas Mavericks,82,50.553659,11.029268,17.908537,18.245122,53.058537,14.065854,22.154878,22.048780
7,Denver Nuggets,82,53.170732,12.815854,26.974390,21.760976,53.312195,10.485366,21.197561,19.553659
8,Detroit Pistons,82,49.223171,10.495122,24.051220,15.860976,51.857317,11.573171,18.639024,19.907317
9,Golden State Warriors,83,56.448193,12.973494,22.509639,20.515663,48.706024,12.912048,25.227711,20.108434


In [289]:
df2018 = pd.read_csv("2018games.csv")
df2018.drop(columns=['Start (ET)', 'Unnamed: 6', 'Unnamed: 7', 'Notes', 'Attend.', 'Notes'], inplace=True)
df2018["MarginHome"] = df2018["PTS.1"] - df2018["PTS"]
df2018.columns = ['Date', 'TeamAway', 'PointsAway', 'TeamHome', 'PointsHome', 'MarginHome']
links = []
for j in df2018.iterrows():
    date = j[1]["Date"]
    team = j[1]["TeamHome"]
    year = date[len(date)-4:]
    month = monthNum[date[4:7]]
    day = date[len(date)-7:len(date)-5]
    if ' ' in day:
        day = '0' + day[1]
    day = day + '0'
    abbrev = teamDict[team]
    link = year + month + day + abbrev
    links.append(link)
df2018['Links'] = links
df2018

,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Links
0,Tue Oct 17 2017,Boston Celtics,99,Cleveland Cavaliers,102,3,201710170CLE
1,Tue Oct 17 2017,Houston Rockets,122,Golden State Warriors,121,-1,201710170GSW
2,Wed Oct 18 2017,Milwaukee Bucks,108,Boston Celtics,100,-8,201710180BOS
3,Wed Oct 18 2017,Atlanta Hawks,117,Dallas Mavericks,111,-6,201710180DAL
4,Wed Oct 18 2017,Charlotte Hornets,90,Detroit Pistons,102,12,201710180DET
5,Wed Oct 18 2017,Brooklyn Nets,131,Indiana Pacers,140,9,201710180IND
6,Wed Oct 18 2017,New Orleans Pelicans,91,Memphis Grizzlies,103,12,201710180MEM
7,Wed Oct 18 2017,Miami Heat,109,Orlando Magic,116,7,201710180ORL
8,Wed Oct 18 2017,Portland Trail Blazers,124,Phoenix Suns,76,-48,201710180PHO
9,Wed Oct 18 2017,Houston Rockets,105,Sacramento Kings,100,-5,201710180SAC


In [291]:
for i, link in enumerate(df2018['Links']):
    if i % 10 == 0:
        print(i)
    url = "https://www.basketball-reference.com/boxscores/" + link + ".html"
    request = requests.get(url)
    if request.status_code != 200:
        print("ERROR")
        print(link)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    comments = soup.find_all(string=lambda text:isinstance(text,Comment))
    for comment in comments:
        comment = BeautifulSoup(str(comment), 'lxml')
        tableFF = comment.find('table', id='four_factors')
        if tableFF:
            break
    all_trs = tableFF.find_all('tr')
    awayTR = all_trs[-2]
    homeTR = all_trs[-1]
    all_tds = awayTR.find_all('td')
    for td in all_tds:
        if td['data-stat'] == 'pace':
            df2018.at[i, 'Pace'] = float(td.text)
        if td['data-stat'] == 'efg_pct':
            df2018.at[i, 'aEFG'] = float(td.text) * 100
        if td['data-stat'] == 'tov_pct':
            df2018.at[i, 'aTOV'] = float(td.text)
        if td['data-stat'] == 'orb_pct':
            df2018.at[i, 'aORB'] = float(td.text)
        if td['data-stat'] == 'ft_rate':
            df2018.at[i, 'aFTR'] = float(td.text) * 100
    all_tds = homeTR.find_all('td')
    for td in all_tds:
        if td['data-stat'] == 'efg_pct':
            df2018.at[i, 'hEFG'] = float(td.text) * 100
        if td['data-stat'] == 'tov_pct':
            df2018.at[i, 'hTOV'] = float(td.text)
        if td['data-stat'] == 'orb_pct':
            df2018.at[i, 'hORB'] = float(td.text)
        if td['data-stat'] == 'ft_rate':
            df2018.at[i, 'hFTR'] = float(td.text) * 100
df2018

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220


,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Links,Pace,aEFG,aTOV,aORB,aFTR,hEFG,hTOV,hORB,hFTR
0,Tue Oct 17 2017,Boston Celtics,99,Cleveland Cavaliers,102,3,201710170CLE,99.3,45.5,9.2,18.0,21.6,48.8,15.3,19.6,25.3
1,Tue Oct 17 2017,Houston Rockets,122,Golden State Warriors,121,-1,201710170GSW,102.0,56.2,10.2,22.2,13.4,63.8,16.0,15.4,23.8
2,Wed Oct 18 2017,Milwaukee Bucks,108,Boston Celtics,100,-8,201710180BOS,97.9,54.6,14.4,15.8,32.9,48.9,10.7,22.0,12.1
3,Wed Oct 18 2017,Atlanta Hawks,117,Dallas Mavericks,111,-6,201710180DAL,98.6,55.9,11.4,30.4,12.8,54.1,13.6,20.0,20.9
4,Wed Oct 18 2017,Charlotte Hornets,90,Detroit Pistons,102,12,201710180DET,98.5,45.9,16.5,7.3,31.5,47.4,7.3,17.0,11.5
5,Wed Oct 18 2017,Brooklyn Nets,131,Indiana Pacers,140,9,201710180IND,113.3,54.3,15.6,25.0,30.9,56.4,10.8,30.4,24.5
6,Wed Oct 18 2017,New Orleans Pelicans,91,Memphis Grizzlies,103,12,201710180MEM,93.3,42.4,15.6,34.8,30.4,47.3,9.5,30.0,17.4
7,Wed Oct 18 2017,Miami Heat,109,Orlando Magic,116,7,201710180ORL,105.2,47.5,10.7,22.0,12.9,52.2,12.0,25.0,24.4
8,Wed Oct 18 2017,Portland Trail Blazers,124,Phoenix Suns,76,-48,201710180PHO,98.7,56.7,13.5,39.5,24.4,35.4,14.2,19.2,14.6
9,Wed Oct 18 2017,Houston Rockets,105,Sacramento Kings,100,-5,201710180SAC,96.3,47.0,12.8,26.7,32.5,52.3,15.5,22.0,9.1


In [310]:
from sklearn.linear_model import LinearRegression

x = []
points = []
k = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model3 = pyGPs.GPR()
model3.setPrior(mean=pyGPs.mean.Zero(), kernel=k)
# create x and y
tempDF = df.copy()
for index, game in df.iterrows():
    aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
    aTeamOff = aTeam[0][2:6]
    aTeamDef = aTeam[0][6:]
    hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
    hTeamOff = hTeam[0][2:6]
    hTeamDef = hTeam[0][6:]
    hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
    aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
    pace = game.Pace
#     hOffDef.append(hTeam[0][1])
#     print(hOffDef)
#     aOffDef.append(aTeam[0][1])
#     print(aOffDef)
    x.append(hTeamOff)
    x.append(aTeamOff)
    points.append(100 * float(game.PointsHome) / pace)
    points.append(100 * float(game.PointsAway) / pace)
# store predictions
# model3.getPosterior(np.array(x), np.array(points), False)
print(points)
lr = LinearRegression()
lr.fit(x, points)
lr.score(x, points)

# tempX = np.array([1,2,3,4,5,6,7,8,9,10]).reshape(-1, 1)
# tempY = np.array([3,6,9,12,15,18,21,24,27,30]).reshape(-1, 1)

# lr.fit(tempX, tempY)
# lr.score(tempX, tempY)

# for index, game in df.iterrows():
#     if index % 50 == 0:
#         print(index)
#     aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
#     aTeamOff = aTeam[0][2:6]
#     aTeamDef = aTeam[0][6:]
#     hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
#     hTeamOff = hTeam[0][2:6]
#     hTeamDef = hTeam[0][6:]
#     hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
#     aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
#     netOffDef = list(map(operator.sub, hOffDef, aOffDef))
#     ym, ys2, fmu, fs2, lp = model.predict(np.array([netOffDef]))
#     predDF.at[index,'PredMean'] = ym[0]
#     predDF.at[index,'PredVar'] = ys2[0]
# tempDF

[117.1171171171171, 88.08808808808809, 101.7293997965412, 131.23092573753814, 124.86187845303867, 114.9171270718232, 118.44660194174757, 113.59223300970874, 125.4826254826255, 116.7953667953668, 120.72434607645874, 114.6881287726358, 111.84210526315789, 107.45614035087719, 99.48186528497409, 110.88082901554404, 95.23809523809524, 99.90662931839402, 101.80275715800637, 114.52810180275716, 96.70987038883351, 102.69192422731805, 97.10743801652893, 116.73553719008265, 114.49579831932773, 95.58823529411764, 110.25145067698259, 95.74468085106382, 111.46496815286623, 105.09554140127388, 105.15873015873017, 113.0952380952381, 103.2967032967033, 112.08791208791209, 103.2064128256513, 94.18837675350701, 105.26315789473685, 113.8560687432868, 113.32633788037776, 86.04407135362015, 93.62139917695472, 99.79423868312757, 105.84958217270194, 113.27762302692665, 108.13397129186603, 105.26315789473684, 96.91160809371671, 100.10649627263045, 104.23452768729642, 96.6340933767644, 126.33832976445396, 108.

0.07593046171226214

In [345]:
x = []
margins = []
# k = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
model = pyGPs.GPR()
model.setPrior(mean=pyGPs.mean.Zero(), kernel=k2)
modelSet = False
# create x and y
for index, game in df.iterrows():
    aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
    aTeamOff = aTeam[0][2:6]
    aTeamDef = aTeam[0][6:]
    hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
    hTeamOff = hTeam[0][2:6]
    hTeamDef = hTeam[0][6:]
    hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
    aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
    netOffDef = list(map(operator.sub, hOffDef, aOffDef))
    x.append(netOffDef)
    margins.append(int(game.MarginHome))
# store predictions
model.optimize(np.array(x),np.array(margins))
model.getPosterior(np.array(x), np.array(margins), False)
for index, game in df2018.iterrows():
    if index % 100 == 0:
        print(index)
    aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
    aTeamOff = aTeam[0][2:6]
    aTeamDef = aTeam[0][6:]
    hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
    hTeamOff = hTeam[0][2:6]
    hTeamDef = hTeam[0][6:]
    hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
    aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
    netOffDef = list(map(operator.sub, hOffDef, aOffDef))
    ym, ys2, fmu, fs2, lp = model.predict(np.array([netOffDef]))
    df2018.at[index,'PredMean'] = ym[0]
    df2018.at[index,'PredVar'] = ys2[0]
df2018

Number of line searches 9


0
100
200
300
400
500
600
700
800
900
1000
1100
1200


,Date,TeamAway,PointsAway,TeamHome,PointsHome,MarginHome,Links,Pace,aEFG,aTOV,aORB,aFTR,hEFG,hTOV,hORB,hFTR,PredMean,PredVar
0,Tue Oct 17 2017,Boston Celtics,99,Cleveland Cavaliers,102,3,201710170CLE,99.3,45.5,9.2,18.0,21.6,48.8,15.3,19.6,25.3,3.100417,187.461397
1,Tue Oct 17 2017,Houston Rockets,122,Golden State Warriors,121,-1,201710170GSW,102.0,56.2,10.2,22.2,13.4,63.8,16.0,15.4,23.8,3.100417,187.461397
2,Wed Oct 18 2017,Milwaukee Bucks,108,Boston Celtics,100,-8,201710180BOS,97.9,54.6,14.4,15.8,32.9,48.9,10.7,22.0,12.1,3.100417,187.461397
3,Wed Oct 18 2017,Atlanta Hawks,117,Dallas Mavericks,111,-6,201710180DAL,98.6,55.9,11.4,30.4,12.8,54.1,13.6,20.0,20.9,3.100417,187.461397
4,Wed Oct 18 2017,Charlotte Hornets,90,Detroit Pistons,102,12,201710180DET,98.5,45.9,16.5,7.3,31.5,47.4,7.3,17.0,11.5,3.100417,187.461397
5,Wed Oct 18 2017,Brooklyn Nets,131,Indiana Pacers,140,9,201710180IND,113.3,54.3,15.6,25.0,30.9,56.4,10.8,30.4,24.5,3.100417,187.461397
6,Wed Oct 18 2017,New Orleans Pelicans,91,Memphis Grizzlies,103,12,201710180MEM,93.3,42.4,15.6,34.8,30.4,47.3,9.5,30.0,17.4,3.100417,187.461397
7,Wed Oct 18 2017,Miami Heat,109,Orlando Magic,116,7,201710180ORL,105.2,47.5,10.7,22.0,12.9,52.2,12.0,25.0,24.4,3.100417,187.461397
8,Wed Oct 18 2017,Portland Trail Blazers,124,Phoenix Suns,76,-48,201710180PHO,98.7,56.7,13.5,39.5,24.4,35.4,14.2,19.2,14.6,3.100417,187.461397
9,Wed Oct 18 2017,Houston Rockets,105,Sacramento Kings,100,-5,201710180SAC,96.3,47.0,12.8,26.7,32.5,52.3,15.5,22.0,9.1,3.100417,187.461397


In [344]:
numRight = 0
for index, row in df2018.iterrows():
    margin = row.MarginHome
    pred = row.PredMean
    if margin > 0 and pred > 0:
        numRight += 1
    if margin < 0 and pred < 0:
        numRight += 1
#     print(pred)
numRight / len(df2018)

0.6170731707317073

In [329]:
x = []
margins = []
k1 = pyGPs.cov.RBF(log_ell=math.log(1/math.sqrt(2)), log_sigma=math.log(1))
k2 = pyGPs.cov.RQ(log_ell=math.log(1), log_sigma=math.log(1), log_alpha=math.log(1))
k3 = pyGPs.cov.Matern(log_ell=math.log(3), d=3, log_sigma=math.log(1))
k4 = pyGPs.cov.Noise(log_sigma=math.log(1))
# k2 = pyGPs.cov.RQ(log_ell=math.log(1), log_sigma=math.log(1), log_alpha=math.log(1))
# k3 = pyGPs.cov.Matern(log_ell=math.log(3), d=3, log_sigma=math.log(1))
# k4 = pyGPs.cov.Noise(log_sigma=math.log(1))
kernels = [k1, k2, k3, k4]
# create x and y
for index, game in df.iterrows():
    aTeam = teamsDF.loc[teamsDF['Team'] == game.TeamAway].values.tolist()
    aTeamOff = aTeam[0][2:6]
    aTeamDef = aTeam[0][6:]
    hTeam = teamsDF.loc[teamsDF['Team'] == game.TeamHome].values.tolist()
    hTeamOff = hTeam[0][2:6]
    hTeamDef = hTeam[0][6:]
    hOffDef = list(map(operator.add, hTeamOff, aTeamDef))
    aOffDef = list(map(operator.add, aTeamOff, hTeamDef))
    netOffDef = list(map(operator.sub, hOffDef, aOffDef))
    x.append(netOffDef)
    margins.append(int(game.MarginHome))
# store predictions
for k in kernels:
    model = pyGPs.GPR()
    model.setPrior(mean=pyGPs.mean.Zero(), kernel=k)
    nlz, post = model.getPosterior(np.array(x), np.array(margins), False)
    print(nlz)

3025868.0245920364
2959026.695527303
2956371.755550771
2956159.8977996144
